In [1]:
import sys
sys.path.append("../../")

import torch
import numpy as np

from riemannian_geometry.computations.riemann_metric import LocalDiagPCA
from models.supervised.mlp.model import MLP
from models.data.sklearn_datasets import MoonDataset, SpiralDataset, BlobsDataset, CirclesDataset
from experiments.visuals.plot_metric import plot_riemann_metric
models_path = "../../models/supervised/mlp/saved_models"
# We want to measure the ``flatness'' of the output space of a neural network
# to then use the identity for sectional curvature as well as providing supporting
# evidence for the manifold hypothesis.

# We will use 3 methods to estimate the Riemann metric taken from https://github.com/georgiosarvanitidis/geometric_ml
# 1. Local PCA
# 2. 

In [2]:
np.random.seed(2)
torch.manual_seed(2)

In [3]:
mode = 'moon'
if mode == 'moon':
    dataset = MoonDataset(n_samples=1000, noise=0.01)
elif mode == 'spiral':
    dataset = SpiralDataset(n_samples=1000, noise=0.01)
elif mode == 'blobs':
    dataset = BlobsDataset(n_samples=1000, noise=0.01)
elif mode == 'circles':
    dataset = CirclesDataset(n_samples=1000, noise=0.01)

In [4]:
model = MLP(2,7,2,2)
model.load_state_dict(torch.load(f'{models_path}/2_wide/mlp_{mode}/model_199.pth'))

<All keys matched successfully>

In [5]:
x_min, x_max = -2, 2
y_min, y_max = -2, 2
h = 0.2
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
xy_grid = np.vstack([xx.ravel(), yy.ravel()]).T
manifold = LocalDiagPCA(xy_grid, sigma=0.05, rho=1e-3)
metric = np.zeros((len(xy_grid), 2))

for indx, coord in enumerate(xy_grid):
    coord = coord.reshape(-1, 1)
    metric[indx] = (manifold.metric_tensor(coord)[0]/1000*h).tolist()


In [6]:
%%capture
labels = [1]*len(xy_grid)
plot_riemann_metric(model, xy_grid, labels, 0, 'figures')
